# Description

It combines all clustering solutions generated into a single consolidated solution using consensus clustering.

# Environment variables

In [1]:
from IPython.display import display

# set numpy n_jobs to 1, since I'll be using n_jobs later
NUMPY_N_JOBS = 1
display(NUMPY_N_JOBS)

1

In [2]:
%env MKL_NUM_THREADS=$NUMPY_N_JOBS
%env OPEN_BLAS_NUM_THREADS=$NUMPY_N_JOBS
%env NUMEXPR_NUM_THREADS=$NUMPY_N_JOBS
%env OMP_NUM_THREADS=$NUMPY_N_JOBS

env: MKL_NUM_THREADS=1
env: OPEN_BLAS_NUM_THREADS=1
env: NUMEXPR_NUM_THREADS=1
env: OMP_NUM_THREADS=1


# Modules loading

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from pathlib import Path

import numpy as np
import pandas as pd

import conf

# Settings

In [5]:
np.random.seed(0)

In [6]:
RANDOM_STATES_ITER = iter(np.random.randint(0, np.iinfo(np.int32).max, size=100000))

In [7]:
# n_init parameter for DeltaSpectralClustering
# a high number should produce more stable final solutions
SC_N_INIT = 50

## Consensus clustering

In [8]:
CLUSTERING_OPTIONS = {}

CLUSTERING_OPTIONS["K_MIN"] = 2
CLUSTERING_OPTIONS["K_MAX"] = 60

display(CLUSTERING_OPTIONS)

{'K_MIN': 2, 'K_MAX': 60}

In [9]:
# output dir for this notebook
RESULTS_DIR = Path(conf.RESULTS["CLUSTERING_DIR"], "consensus_clustering").resolve()
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

display(RESULTS_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/clustering/consensus_clustering')

# Load ensemble

In [10]:
output_file = Path(RESULTS_DIR, "ensemble.npy").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/clustering/consensus_clustering/ensemble.npy')

In [11]:
full_ensemble = np.load(output_file)

In [12]:
display(full_ensemble.shape)

(4428, 3752)

# Load ensemble coassociation distance matrix

In [13]:
output_file = Path(RESULTS_DIR, "ensemble_coassoc_matrix.npy").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/clustering/consensus_clustering/ensemble_coassoc_matrix.npy')

In [14]:
ensemble_coassoc_matrix = np.load(output_file)

In [15]:
display(ensemble_coassoc_matrix.shape)

(3752, 3752)

In [16]:
display(ensemble_coassoc_matrix)

array([[0.        , 0.31497143, 0.328     , ..., 0.47951098, 0.41141688,
        0.64433341],
       [0.31497143, 0.        , 0.04153355, ..., 0.67868553, 0.6486054 ,
        0.67496005],
       [0.328     , 0.04153355, 0.        , ..., 0.66408033, 0.64220393,
        0.67701438],
       ...,
       [0.47951098, 0.67868553, 0.66408033, ..., 0.        , 0.48558456,
        0.51122195],
       [0.41141688, 0.6486054 , 0.64220393, ..., 0.48558456, 0.        ,
        0.66749943],
       [0.64433341, 0.67496005, 0.67701438, ..., 0.51122195, 0.66749943,
        0.        ]])

# Consensus clustering

In [17]:
from concurrent.futures import ProcessPoolExecutor, as_completed

from tqdm import tqdm

from clustering.ensembles.utils import (
    run_method_and_compute_agreement,
)
from clustering.ensembles.eac import (
    eac_single_coassoc_matrix,
    eac_complete_coassoc_matrix,
    eac_average_coassoc_matrix,
)
from clustering.ensembles.spectral import scc

Define spectral consensus clustering methods with delta values found in pre-analysis:

In [18]:
def scc_020(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.20,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )


def scc_025(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.25,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )


def scc_030(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.30,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )


def scc_050(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.50,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )

In [19]:
all_consensus_methods = set(
    (
        eac_single_coassoc_matrix,
        eac_complete_coassoc_matrix,
        eac_average_coassoc_matrix,
        scc_020,
        scc_025,
        scc_030,
        scc_050,
    )
)
display(all_consensus_methods)

{<function clustering.ensembles.eac.eac_average_coassoc_matrix(coassoc_matrix, k, **kwargs)>,
 <function clustering.ensembles.eac.eac_complete_coassoc_matrix(coassoc_matrix, k, **kwargs)>,
 <function clustering.ensembles.eac.eac_single_coassoc_matrix(coassoc_matrix, k, **kwargs)>,
 <function __main__.scc_020(coassoc_distance_matrix, k, **kwargs)>,
 <function __main__.scc_025(coassoc_distance_matrix, k, **kwargs)>,
 <function __main__.scc_030(coassoc_distance_matrix, k, **kwargs)>,
 <function __main__.scc_050(coassoc_distance_matrix, k, **kwargs)>}

In [20]:
consensus_results = []

with ProcessPoolExecutor(max_workers=conf.GENERAL["N_JOBS"]) as executor:
    tasks = {
        executor.submit(
            run_method_and_compute_agreement,
            m,
            ensemble_coassoc_matrix,
            full_ensemble,
            k,
            n_init=SC_N_INIT,
            random_state=next(RANDOM_STATES_ITER),
        ): (m.__name__, k)
        for m in all_consensus_methods
        for k in range(CLUSTERING_OPTIONS["K_MIN"], CLUSTERING_OPTIONS["K_MAX"] + 1)
    }

    for future in tqdm(as_completed(tasks), total=len(tasks), disable=False, ncols=100):
        method_name, k = tasks[future]
        part, performance_values = future.result()

        method_results = {
            "method": method_name,
            "partition": part,
            "k": k,
        }
        method_results.update(performance_values)

        consensus_results.append(method_results)

  0%|                                                                       | 0/413 [00:00<?, ?it/s]

  0%|▏                                                            | 1/413 [00:24<2:51:00, 24.90s/it]

  0%|▎                                                            | 2/413 [00:25<2:00:43, 17.62s/it]

  1%|▍                                                            | 3/413 [00:28<1:31:06, 13.33s/it]

  1%|▌                                                            | 4/413 [00:53<1:53:20, 16.63s/it]

...

 99%|██████████████████████████████████████████████████████████▌| 410/413 [1:37:21<00:53, 17.99s/it]

100%|██████████████████████████████████████████████████████████▋| 411/413 [1:37:37<00:34, 17.48s/it]

100%|██████████████████████████████████████████████████████████▊| 412/413 [1:37:56<00:17, 17.71s/it]

100%|███████████████████████████████████████████████████████████| 413/413 [1:38:14<00:00, 17.99s/it]

100%|███████████████████████████████████████████████████████████| 413/413 [1:38:14<00:00, 14.27s/it]

In [21]:
consensus_results = pd.DataFrame(consensus_results)

In [22]:
display(consensus_results.shape)

(413, 12)

In [23]:
consensus_results.head()

,method,partition,k,ari_mean,ari_median,ari_std,ami_mean,ami_median,ami_std,nmi_mean,nmi_median,nmi_std
0,scc_030,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,0.170172,0.016107,0.280429,0.182083,0.073948,0.241458,0.184787,0.076497,0.240941
1,scc_030,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3,0.170593,0.016107,0.280689,0.198427,0.080792,0.247718,0.202181,0.083653,0.246786
2,scc_030,"[0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 3, 0, 3, 0, ...",4,0.162009,0.086061,0.197582,0.227147,0.204954,0.145220,0.232457,0.211766,0.145033
3,scc_030,"[1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 4, 4, 1, 1, 1, ...",5,0.163732,0.087616,0.198028,0.238870,0.217941,0.145885,0.244974,0.225387,0.145597
4,scc_030,"[0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 5, 5, 0, 5, 0, ...",6,0.164568,0.088147,0.199509,0.250476,0.231739,0.150811,0.257361,0.238495,0.150334


## Testing

In [24]:
assert not consensus_results.isna().any().any()

In [25]:
# check that the number of clusters in the partitions are the expected ones
_real_k_values = consensus_results["partition"].apply(lambda x: np.unique(x).shape[0])
display(_real_k_values)
assert np.all(consensus_results["k"].values == _real_k_values.values)

0       2
1       3
2       4
3       5
4       6
       ..
408    56
409    57
410    58
411    59
412    60
Name: partition, Length: 413, dtype: int64

## Save

In [26]:
output_file = Path(RESULTS_DIR, "consensus_clustering_runs.pkl").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/clustering/consensus_clustering/consensus_clustering_runs.pkl')

In [27]:
consensus_results.to_pickle(output_file)